In [2]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

In [7]:
def naive_bayes(dict,text):
    # Preparação dos dados de treinamento
    texts = text
    labels = list(dict.keys())
    # Criação do vetorizador usando as palavras-chave dos equipamentos
    vectorizer = CountVectorizer(vocabulary=[keyword for keywords in dict.values() for keyword in keywords], ngram_range=(1, 2))
    # Vetorização das descrições dos problemas
    X_train = vectorizer.transform(texts)
    # Criação do modelo MultinomialNB
    model = MultinomialNB()
    # Treinamento do modelo
    model.fit(X_train, labels)

In [6]:
from re import sub
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from spacy.lang.pt.stop_words import STOP_WORDS
from nltk.corpus import wordnet
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer

In [13]:
def remove_num(text):
    text = sub(r'\d+', '', text)
    text = sub(r'\s+', ' ',text)
    return text

def remove_punct(text):
    text = sub(r"[!#$%&'()*+,-./:;<=>?@[^_`{|}~]+", ' ',text)
    text = sub(r'\s+', ' ',text)
    return text

def get_keywords(text):
    tokens = word_tokenize(text)
    keywords = []
    for word in tokens:
        word = word.lower()
        if word not in stopwords.words('portuguese') or word.lower() not in STOP_WORDS:
            keywords.append(word)
    return keywords

def get_synonyms(text):
    tokens = word_tokenize(text)
    synonyms = []
    for word in tokens:
        for syn in wordnet.synsets(word, lang="por"):
            for lemma in syn.lemmas(lang="por"):
                synonyms.append(lemma.name())
    return synonyms

def preprocess_lemma(text):
    lemmatizer = WordNetLemmatizer()
    tokens = word_tokenize(text)
    lemmas = []
    for token in tokens:
        lemmas.append(lemmatizer.lemmatize(token))
    lemmas = ' '.join(lemmas)
    return lemmas

def remove_accent(text):
    text = sub('[áàãâä]', 'a', sub('[éèêë]', 'e', sub('[íìîï]', 'i', sub('[óòõôö]', 'o', sub('[úùûü]', 'u', text)))))
    text = sub(r'\s+', ' ',text)
    return text

def preprocess_stem(text):
    stemmer = SnowballStemmer("portuguese")
    tokens = word_tokenize(text)
    stems = []
    for token in tokens:
        stems.append(stemmer.stem(token))
    stems = ' '.join(stems)
    return stems

### Preparando o dataframe

In [9]:
import pandas as pd

In [11]:
df = pd.read_excel(r'..\portugues\pt_troubleshooting.xlsx')

In [12]:
class_equipament = []
for equip in df['equipament']:
    if equip not in class_equipament and equip != 'Motores elétricos (dc)':
        class_equipament.append(equip)
class_equipament

['bombas',
 'rolamentos',
 'válvulas',
 'acionamentos por corrente',
 'caixas de engrenagens',
 'Sistemas de óleo lubrificante',
 'Acionamentos por correia em V',
 'Sistemas de ventiladores',
 'Purgadores de vapor',
 'Motores elétricos',
 'Contatos elétricos',
 'Disjuntores elétricos de caixa moldada',
 'Circuito magnético',
 'Circuito dielétrico']

In [20]:
grupo_equipamentos = ['bombas',
                     'rolamentos',
                     'válvulas',
                     'acionamentos por corrente',
                     'caixas de engrenagens',
                     'Sistemas de óleo lubrificante',
                     'Acionamentos por correia em V',
                     'Sistemas de ventiladores',
                     'Purgadores de vapor',
                     'Motores elétricos']

grupo_componentes = ['Contatos elétricos',
                     'Disjuntores elétricos de caixa moldada',
                     'Circuito magnético',
                     'Circuito dielétrico']

In [31]:
from fuzzywuzzy import fuzz
from nltk.tokenize import word_tokenize

def similares(frase, lista_classes):
    palavras_similares = []
    palavras_frase = word_tokenize(frase.lower())

    for palavra in palavras_frase:
        similaridades = []
        for classe in lista_classes:
            similaridade = fuzz.ratio(palavra, classe)
            similaridades.append((classe, similaridade))

        similaridades = sorted(similaridades, key=lambda x: x[1], reverse=True)
        if similaridades and similaridades[0][1] >= 80:
            palavra_similar = {
                "palavra": palavra,
                "classe": similaridades[0][0],
                "similaridade": similaridades[0][1]
            }
        else:
            palavra_similar = {
                "palavra": palavra,
                "classe": None,
                "similaridade": 0
            }
        
        palavras_similares.append(palavra_similar)

    return palavras_similares


In [34]:
texto = "Bomba muito quente e piscando. Bomba agitando o fluido e não descarregando a água após um surto inicial e continuando a recircular."
lista_classes = ['bombas',
                     'rolamentos',
                     'válvulas',
                     'acionamentos por corrente',
                     'caixas de engrenagens',
                     'Sistemas de óleo lubrificante',
                     'Acionamentos por correia em V',
                     'Sistemas de ventiladores',
                     'Purgadores de vapor',
                     'Motores elétricos']

resultado = similares(texto, lista_classes)
resultado

[{'palavra': 'bomba', 'classe': 'bombas', 'similaridade': 91},
 {'palavra': 'muito', 'classe': None, 'similaridade': 0},
 {'palavra': 'quente', 'classe': None, 'similaridade': 0},
 {'palavra': 'e', 'classe': None, 'similaridade': 0},
 {'palavra': 'piscando', 'classe': None, 'similaridade': 0},
 {'palavra': '.', 'classe': None, 'similaridade': 0},
 {'palavra': 'bomba', 'classe': 'bombas', 'similaridade': 91},
 {'palavra': 'agitando', 'classe': None, 'similaridade': 0},
 {'palavra': 'o', 'classe': None, 'similaridade': 0},
 {'palavra': 'fluido', 'classe': None, 'similaridade': 0},
 {'palavra': 'e', 'classe': None, 'similaridade': 0},
 {'palavra': 'não', 'classe': None, 'similaridade': 0},
 {'palavra': 'descarregando', 'classe': None, 'similaridade': 0},
 {'palavra': 'a', 'classe': None, 'similaridade': 0},
 {'palavra': 'água', 'classe': None, 'similaridade': 0},
 {'palavra': 'após', 'classe': None, 'similaridade': 0},
 {'palavra': 'um', 'classe': None, 'similaridade': 0},
 {'palavra': 's